In [4]:
# Load data 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from scipy.misc import imresize
import matplotlib.pyplot as plt
%matplotlib inline

# fix random seed for reproducibility
np.random.seed(7)
# Load MNIST dataset
from keras.datasets import mnist
(Xtrain, Ytrain), (Xtest, Ytest) = mnist.load_data()

(ntrain, xdim, ydim) = Xtrain.shape
ntest = Xtest.shape[0]

# split train data in two
X_pr = Xtrain[30000:60000, :, :]
Y_pr = Ytrain[30000:60000]

Xtrain = Xtrain[0:30000, :, :];
Ytrain = Ytrain[0:30000]

# downsample
factor = 0.25

Xtrain_down = np.ones((Xtrain.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtrain.shape[0]):
    Xtrain_down[i, :, :] = imresize(Xtrain[i,:,:], factor)

Xtest_down = np.ones((Xtest.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(Xtest.shape[0]):
    Xtest_down[i,:,:] = imresize(Xtest[i,:,:], factor)

X_pr_down = np.ones((X_pr.shape[0], int(xdim*factor), int(ydim*factor)))
for i in range(X_pr.shape[0]):
    X_pr_down[i,:,:] = imresize(X_pr[i,:,:], factor)
    
# # *** VECTORIZE IMAGES ***
Xtrain_down = Xtrain_down.reshape(Xtrain_down.shape[0], int(xdim*factor)**2).astype('float32') / 255
Xtest_down = Xtest_down.reshape(ntest, int(xdim*factor)**2).astype('float32') / 255
X_pr_down = X_pr_down.reshape(X_pr_down.shape[0], int(xdim*factor)**2).astype('float32') / 255

Ytrain_cat = np_utils.to_categorical(Ytrain, 10)
Ytest_cat = np_utils.to_categorical(Ytest, 10)


Using TensorFlow backend.


In [ ]:
import tensorflow as tf
# *** TRAIN A FULLY-CONNECTED NN WITH THREE HIDDEN LAYERS ***
# create model
model = Sequential()
model.add(Dense(100, input_shape=(int(xdim*factor)**2,), activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(100, activation='relu'))
model.add(Dense(10, activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Fit the model
with tf.device('/gpu:1'):
    model.fit(Xtrain_down, Ytrain_cat, nb_epoch=50, batch_size=16)

# evaluate the model
scores = model.evaluate(Xtest_down, Ytest_cat, verbose=0)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

inp = model.input                                           # input placeholder
outputs = [layer.output for layer in model.layers]          # all layer outputs
functor = K.function([inp]+ [K.learning_phase()], outputs ) # evaluation function
layer_outs = functor([X_pr_down, 1.])

import pickle
with open('layer_outs.dat','wb') as f:
    pickle.dump(layer_outs, f)
f.close()

Epoch 1/50
30000/30000 [==============================] - 8s - loss: 0.4526 - acc: 0.8623     
Epoch 2/50
30000/30000 [==============================] - 6s - loss: 0.2237 - acc: 0.9292     
Epoch 3/50
30000/30000 [==============================] - 6s - loss: 0.1769 - acc: 0.9438     
Epoch 4/50
30000/30000 [==============================] - 6s - loss: 0.1527 - acc: 0.9518     
Epoch 5/50
30000/30000 [==============================] - 6s - loss: 0.1353 - acc: 0.9569     
Epoch 6/50
14608/30000 [=============>................] - ETA: 3s - loss: 0.1186 - acc: 0.9613

In [1]:
# subsample and do xgboost regression
from copy import copy, deepcopy
from RE_PartialRecData import RE_PartialRecData

import os
import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier

params = {}
# use softmax multi-class classification 'multi:softmax'
# use linear regression 'reg:linear'
params['objective'] = 'reg:linear'
# scale weight of positive examples
params['eta'] = 0.4
params['max_depth'] = 5
params['silent'] = 1
params['nthread'] = 4
# params['num_class'] = 10
num_round=5


# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = range(1,100,5)
nSubnetSize = len(subnetSize)
# which layer?
iLayer=0;
# how many samples per recording?
#nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)
nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)*100
# how many iterations
nIterations = 50

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]
rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])

for ss in range(nSubnetSize):
    nLayerNeurons = subnetSize[ss]
    print(subnetSize[ss])
    for it in range(nIterations):
        # copy data
        layer_outputs = deepcopy(layer_outs)
        # subsample
        X_subsample, Y_subsample = RE_PartialRecData2(layer_ouputs[iLayer], layer_outputs[oLayer], nLayerNeurons, nRecordings, nSamples[ss])
        #print('# nan neurons: ',np.count_nonzero(np.isnan(X_subsample[:3000,:]).sum(axis=0)))
        # prepare data for xgboost
        for iN in range(nOutNeurons):
            #print('#neuron, #iteratin, subnetsize: ', iN,it,subnetSize[ss])
            xg_train  = xgb.DMatrix(X_subsample, label=Y_subsample[:, iN])
            xg_test   = xgb.DMatrix(layer_outs_test[0], label=layer_outs_test[3][:,iN])
            watchlist = [(xg_train, 'train'), (xg_test, 'test')]
            # train XGboost
            bst = xgb.train(params, xg_train, num_round, watchlist, verbose_eval=False)
            # get predictions
            pred = bst.predict(xg_test)
            rmses[it,iN,ss] = np.sqrt(np.mean(np.square([(pred[i] - layer_outs_test[3][:,iN][i]) 
                                         for i in range(len(layer_outs_test[3][:,1]))])))
            #print ('predicting, RMSE=%f' %rmses[it, iN, ss])

            

# save the rmse's
import pickle

with open('RMSE_layer1.dat','wb') as f:
    pickle.dump(rmses, f)
f.close()

/opt/anaconda/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


NameError: name 'np' is not defined

In [ ]:
import pickle

with open('RMSE_layer1.dat','r') as f:
    rmses = pickle.load(f)
f.close()

import matplotlib.pyplot as pl
%matplotlib inline
import numpy as np

fig=pl.figure(figsize=(10,6))
ax1 = fig.add_subplot(111)
ax1.set_xlim([0, 100])
ax2 = ax1.twiny()

x = subnetSize;
y = np.mean(np.mean(rmses, axis=1), axis=0)
error = np.std(np.mean(rmses, axis=1), axis=0)
bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

pl.plot(x, y, 'k-')
horiz_line_data = np.array([bl for i in xrange(len(x))])
pl.plot(x, horiz_line_data, 'k--') 
pl.fill_between(x, y-error, y+error, alpha=0.2, facecolor='#808080')

ax1.set_xlabel('# observed neurons on Layer 1 (out of 100)', fontsize=18)
ax2.set_xlabel('Samples per recording',  fontsize=16)

new_tick_locations =range(1, 100, 20)
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(nSamples[range(0, 20, 4)])


# ax2.set_xticks(nSamples)
# ax2.set_xticklabels(nSamples[range(0, 5, 20)])
ax1.set_ylabel('RMSE', fontsize=18)
ax1.set_ylim([0, .35])

pl.text(90,0.3, 'baseline')

# #pl.title('%d recordings'%(nRecordings), fontsize=18)
# #pl.title(nRecordings ' recordings ')
#pl.show()
pl.draw()
pl.savefig('RMSEs_Layer_1.pdf', format='pdf')#, dpi=1000)
pl.savefig('RMSEs_Layer_1.eps', format='eps', dpi=1000)
pl.savefig('RMSEs_Layer_1.png', format='png')



In [ ]:
# subsample and do xgboost regression
from copy import copy, deepcopy
from RE_PartialRecData import RE_PartialRecData

import os
import xgboost as xgb
import sklearn
from xgboost.sklearn import XGBClassifier
import time


params = {}
# use softmax multi-class classification 'multi:softmax'
# use linear regression 'reg:linear'
params['objective'] = 'reg:linear'
# scale weight of positive examples
params['eta'] = 0.4
params['max_depth'] = 5
params['silent'] = 1
params['nthread'] = 4
# params['num_class'] = 10
num_round=5
# how many recordings?
nRecordings = 10
# how many neurons from the firs hidden layer?
subnetSize = range(1,100,5)
nSubnetSize = len(subnetSize)
# which layers?
# how many samples per recording?
nSamples = np.divide(int(X_pr_down.shape[0]/nRecordings),subnetSize)
# how many iterations
nIterations = 50

# baseline prediction error
#bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

oLayer = len(layer_outs)-1  # index of output layer
nOutNeurons = layer_outs[oLayer].shape[1]
rmses = np.zeros([nIterations, nOutNeurons, nSubnetSize])

for ss in range(nSubnetSize):
    nLayerNeurons = [0, subnetSize[ss], 0, 10]
    start = time.time()

    for it in range(nIterations):
        # copy data
        layer_outputs = deepcopy(layer_outs)
        # subsample
        X_subsample, Y_subsample = RE_PartialRecData(layer_outputs, nLayerNeurons, nRecordings, nSamples[ss])
        #print('# nan neurons: ',np.count_nonzero(np.isnan(X_subsample[:3000,:]).sum(axis=0)))
        # prepare data for xgboost
        for iN in range(nOutNeurons):
            #print('#neuron, #iteratin, subnetsize: ', iN,it,subnetSize[ss])
            xg_train  = xgb.DMatrix(X_subsample, label=Y_subsample[:, iN])
            xg_test   = xgb.DMatrix(layer_outs_test[1], label=layer_outs_test[3][:,iN])
            watchlist = [(xg_train, 'train'), (xg_test, 'test')]
            # train XGboost
            bst = xgb.train(params, xg_train, num_round, watchlist, verbose_eval=False)
            # get predictions
            pred = bst.predict(xg_test)
            rmses[it,iN,ss] = np.sqrt(np.mean(np.square([(pred[i] - layer_outs_test[3][:,iN][i]) 
                                         for i in range(len(layer_outs_test[3][:,1]))])))
            # run your code
    end = time.time()
    elapsed = end - start
    print(subnetSize[ss],elapsed,X_subsample.shape[0])

# save the rmse's
import pickle
with open('RMSE_layer2.dat','wb') as f:
    pickle.dump(rmses, f)
f.close()

In [ ]:
import pickle

with open('RMSE_layer2.dat','r') as f:
    rmses = pickle.load(f)
f.close()

import matplotlib.pyplot as pl
%matplotlib inline
import numpy as np

fig=pl.figure(figsize=(10,6))
ax1 = fig.add_subplot(111)
ax1.set_xlim([0, 100])
ax2 = ax1.twiny()

x = subnetSize;
y = np.mean(np.mean(rmses, axis=1), axis=0)
error = np.std(np.mean(rmses, axis=1), axis=0)
bl = np.std(layer_outs_test[oLayer]-np.mean(layer_outs_test[oLayer]));

pl.plot(x, y, 'k-')
horiz_line_data = np.array([bl for i in xrange(len(x))])
pl.plot(x, horiz_line_data, 'k--') 
pl.fill_between(x, y-error, y+error, alpha=0.2, facecolor='#808080')

ax1.set_xlabel('# observed neurons on Layer 2 (out of 100)', fontsize=16)
ax2.set_xlabel('Samples per recording',  fontsize=18)

new_tick_locations =range(1, 100, 20)
ax2.set_xlim(ax1.get_xlim())
ax2.set_xticks(new_tick_locations)
ax2.set_xticklabels(nSamples[range(0, 20, 4)])


# ax2.set_xticks(nSamples)
# ax2.set_xticklabels(nSamples[range(0, 5, 20)])
ax1.set_ylabel('RMSE', fontsize=18)
ax1.set_ylim([0, .35])

pl.text(90,0.3, 'baseline')

# #pl.title('%d recordings'%(nRecordings), fontsize=18)
# #pl.title(nRecordings ' recordings ')
pl.draw()
pl.savefig('RMSEs_Layer_2.pdf', format='pdf')#, dpi=1000)
pl.savefig('RMSEs_Layer_2.png', format='png')




In [ ]:
nSamples[[1, 6, 10]]